In [2]:
import os, random, shutil
import numpy as np
import pandas as pd
import PIL
#import keras
import itertools
from PIL import Image

import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 
from tqdm import tqdm
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import BatchNormalization
#from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard
import io
import time
import datetime
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib

## Model building

In [3]:
batch_size_train = 25
num_classes= 4
IMAGE_SIZE=[200,200]
classes_required = ['Aviator','Oval','Rectangle','Wayfarer']

In [4]:
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             #shear_range = 0.2,
                             #zoom_range = 0.2
                            )

In [5]:
#Input the Training Data

train_path = 'D:/My ML Projects/lenskart task/sub-train/Non-Power Reading'
train_batches = datagen.flow_from_directory(train_path, target_size=(200,200), classes=classes_required, batch_size=batch_size_train)
type(train_batches)

Found 5012 images belonging to 4 classes.


tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [6]:
batchX, batchy = train_batches.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(25, 200, 200, 3), min=0.000, max=1.000


In [7]:
train_imgs, train_labels = next(train_batches)

In [8]:
train_imgs[0]

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [9]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 64} ) 
sess = tf.compat.v1.Session(config=config) 
#keras.backend.set_session(sess)
tf.compat.v1.keras.backend.set_session(
    sess
)
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.333)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

### Model Architecture

In [10]:
model=Sequential()
model.add(Conv2D(16,kernel_size=(3,3), activation="relu" ,input_shape=IMAGE_SIZE + [3], padding='same'))

model.add(Conv2D(32, kernel_size=(3,3), activation="relu",padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.30))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization()) 
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      4640      
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0

In [12]:
# Complie the model
opt = Adadelta(learning_rate=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(train_batches, steps_per_epoch=25, epochs=100, verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
25/25 [==============================] - 8s 337ms/step - loss: 2.0341 - accuracy: 0.2336
Epoch 2/100
25/25 [==============================] - 8s 325ms/step - loss: 2.0007 - accuracy: 0.2656
Epoch 3/100
25/25 [==============================] - 8s 310ms/step - loss: 1.9755 - accuracy: 0.2624
Epoch 4/100
25/25 [==============================] - 8s 309ms/step - loss: 1.9007 - accuracy: 0.2720
Epoch 5/100
25/25 [==============================] - 8s 305ms/step - loss: 1.8175 - accuracy: 0.2864
Epoch 6/100
25/25 [==============================] - 8s 302ms/step - loss: 1.7483 - accuracy: 0.3136
Epoch 7/100
25/25 [==============================] - 7s 297ms/step - loss: 1.6984 - accuracy: 0.3392
Epoch 8/100
25/25 [==============================] - 7s 281ms/step - loss: 1.6196 - accuracy: 0.3616
Epoch 9/100
25/25 [==============================] - 7s 277ms/step - loss: 1.6776 - accuracy: 0.3584
Epoch 10/100
25

25/25 [==============================] - 11s 457ms/step - loss: 0.7916 - accuracy: 0.6846
Epoch 77/100
25/25 [==============================] - 10s 407ms/step - loss: 0.8492 - accuracy: 0.6560
Epoch 78/100
25/25 [==============================] - 9s 347ms/step - loss: 0.8568 - accuracy: 0.6416
Epoch 79/100
25/25 [==============================] - 8s 339ms/step - loss: 0.8226 - accuracy: 0.6480
Epoch 80/100
25/25 [==============================] - 8s 334ms/step - loss: 0.8050 - accuracy: 0.6656
Epoch 81/100
25/25 [==============================] - 9s 377ms/step - loss: 0.8257 - accuracy: 0.6416
Epoch 82/100
25/25 [==============================] - 9s 360ms/step - loss: 0.8098 - accuracy: 0.6688
Epoch 83/100
25/25 [==============================] - 8s 333ms/step - loss: 0.8089 - accuracy: 0.6672
Epoch 84/100
25/25 [==============================] - 8s 328ms/step - loss: 0.8061 - accuracy: 0.6560
Epoch 85/100
25/25 [==============================] - 9s 341ms/step - loss: 0.7997 - accuracy

In [14]:
model.save("model_NonPowerReading_25_batch.h5")
print("Saved model to disk")

Saved model to disk
